In [ ]:
! pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.8 MB/s eta 0:00:00


### Import libraries

In [ ]:
import os
import torch
from torcheval.metrics.functional import multiclass_accuracy, multiclass_auroc, multiclass_recall
import torch.nn as nn
from torch.optim import Adam
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path = "/content/drive/MyDrive/Deep_learning_course/Text_processing/Embedding/"
nltk_path = "/content/drive/MyDrive/Deep_learning_course/Text_processing/"
nltk_folder_path = "/content/lmtools/"
data_path = "/content/data/"
file_path = os.path.join(data_path, "reviews_train.csv")

In [ ]:
external_lmtool_path = os.path.join(nltk_path, "nltk_data.zip")
local_lmtool_path = nltk_folder_path
!unzip -q $external_lmtool_path -d $local_lmtool_path

In [ ]:
import nltk
nltk_path = os.path.join(nltk_folder_path, "nltk_data")
nltk.data.path.insert(0, nltk_path)

In [ ]:
lib_path = os.path.join(root_path, "src")
import sys
sys.path.append(lib_path)
from embedding_dictionary import EmbeddingDictionary
from embedding_sequence import SequenceFromText
# from data_set import DatasetReviews
from data_loader import dataloader_reviews_train_eval
from fit_func import FitTrainEval
from cbow_model import CBOW

### Create dictionary

In [ ]:
external_data_pat = os.path.join(root_path, "data/")
local_data_path = data_path
!cp -r $external_data_pat -d $local_data_path

In [ ]:
embedding_maker = EmbeddingDictionary(
    text_path=file_path,
    unk_cutoff=3
)

In [ ]:
w2v_dict = embedding_maker.create_embedding_dict()

In [ ]:
with open(os.path.join(external_data_pat, "w2v_dict.pkl"), "wb") as f:
    pickle.dump(w2v_dict, f)

### Create sequences

In [ ]:
hwc = 4

In [ ]:
sequence_maker = SequenceFromText(
    text_path=file_path,
    word_dictionary=w2v_dict,
    half_word_context=hwc
)

In [ ]:
sequences = sequence_maker.create_sequence()

### Load data

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# data parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vocab_size = len(w2v_dict)
batch_size = 10_000
num_workers = 8
# model parameters
model_load = "cbow_7.pkl"
model_save = "cbow_8.pkl"
model_load = os.path.join(root_path, "Models/", model_load)
model_path = os.path.join(root_path, "Models/", model_save)
metrics = [multiclass_accuracy]
EPOCHS = 500

In [ ]:
data_train, data_eval = dataloader_reviews_train_eval(
    sequences=sequences,
    vocab_size=vocab_size,
    batch_size=batch_size,
    num_workers=num_workers,
    device=device
)

### Train models

In [ ]:
model = CBOW(
    vocab_size=vocab_size,
    embedding_dim=300,
    padd_idx=w2v_dict["unk"]
)
# model.load_state_dict(torch.load(model_load))
model = model.to(device)

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-3)

In [ ]:
optimization = FitTrainEval(
    model=model,
    loss=loss_fn,
    optimizer=optimizer,
    metrics=metrics,
    path_save=model_path,
    device=device
)

In [ ]:
trained = optimization.fit_train_eval(
    data_train=data_train,
    data_eval=data_eval,
    epochs=EPOCHS,
)